In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load data
base_dir = Path('../')
processed_data_path = base_dir / 'data/processed/modified_dataset.csv'
raw_data_path = base_dir / 'data/raw/nepal_economic_data.csv'

In [3]:
# Load processed data if available, otherwise load raw data
if processed_data_path.exists():
    df = pd.read_csv(processed_data_path)
    print("Using processed dataset")
else:
    df = pd.read_csv(raw_data_path)
    print("Using raw dataset")

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

Using processed dataset


In [4]:
# 1. Plot Inflation Components
def plot_inflation_components():
    """Plot main inflation components using plotly"""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Inflation Components Over Time', 'Month-over-Month Changes'],
        vertical_spacing=0.15
    )
    
    # Components
    components = ['overall_index', 'food_and_beverage', 'non_food_and_services']
    colors = ['blue', 'green', 'red']
    names = ['Overall', 'Food & Beverage', 'Non-food & Services']
    
    # Plot indices
    for comp, color, name in zip(components, colors, names):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df[comp],
                name=name,
                line=dict(color=color)
            ),
            row=1, col=1
        )
    
    # Plot MoM changes
    mom_components = [comp + '_mom' for comp in components]
    for comp, color, name in zip(mom_components, colors, names):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df[comp],
                name=f"{name} MoM",
                line=dict(color=color)
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        height=800,
        title_text="Nepal Inflation Components Analysis",
        template="plotly_white",
        showlegend=True
    )
    
    fig.update_yaxes(title_text="Index Value", row=1, col=1)
    fig.update_yaxes(title_text="Percent Change (%)", row=2, col=1)
    
    fig.show()

plot_inflation_components()

In [5]:
# 2. Plot Seasonal Patterns
def plot_seasonal_patterns():
    """Analyze seasonal patterns using plotly"""
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Monthly Pattern of Overall Inflation', 
                       'Yearly Trend of Overall Inflation']
    )
    
    # Monthly patterns
    monthly_avg = df.groupby(df.index.month)['overall_index'].mean()
    monthly_std = df.groupby(df.index.month)['overall_index'].std()
    
    fig.add_trace(
        go.Bar(
            x=monthly_avg.index,
            y=monthly_avg,
            error_y=dict(type='data', array=monthly_std),
            name="Monthly Pattern"
        ),
        row=1, col=1
    )
    
    # Yearly trends
    yearly_avg = df.groupby(df.index.year)['overall_index'].mean()
    yearly_std = df.groupby(df.index.year)['overall_index'].std()
    
    fig.add_trace(
        go.Bar(
            x=yearly_avg.index,
            y=yearly_avg,
            error_y=dict(type='data', array=yearly_std),
            name="Yearly Pattern"
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        height=500,
        title_text="Seasonal Patterns in Nepal's Inflation",
        template="plotly_white",
        showlegend=True
    )
    
    fig.show()

plot_seasonal_patterns()

In [6]:
# 3. Plot Correlation Analysis
def plot_correlation_analysis():
    """Plot correlation matrix using plotly"""
    key_features = [
        'overall_index', 'food_and_beverage', 'non_food_and_services',
        'broad_money_m2', 'policy_rate', 'indian_cpi', 
        'usd', 'crude_oil_prices'
    ]
    
    corr_matrix = df[key_features].corr()
    
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix,
        x=key_features,
        y=key_features,
        colorscale='RdBu',
        zmid=0,
        text=np.round(corr_matrix, 2),
        texttemplate='%{text}',
        textfont={"size": 10},
        hoverongaps=False
    ))
    
    fig.update_layout(
        title="Correlation Matrix of Key Economic Indicators",
        template="plotly_white",
        height=600
    )
    
    fig.show()

plot_correlation_analysis()

In [7]:
# 4. Plot Economic Drivers
def plot_economic_drivers():
    """Plot economic relationships using plotly"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Money Supply vs Inflation',
            'Exchange Rate vs Inflation',
            'Indian CPI vs Nepal Inflation',
            'Oil Prices vs Inflation'
        ]
    )
    
    # Money Supply vs Inflation
    fig.add_trace(
        go.Scatter(
            x=df['broad_money_m2'],
            y=df['overall_index'],
            mode='markers',
            name='Money Supply',
            marker=dict(color='blue', opacity=0.6)
        ),
        row=1, col=1
    )
    
    # Exchange Rate vs Inflation
    fig.add_trace(
        go.Scatter(
            x=df['usd'],
            y=df['overall_index'],
            mode='markers',
            name='Exchange Rate',
            marker=dict(color='red', opacity=0.6)
        ),
        row=1, col=2
    )
    
    # Indian CPI vs Nepal Inflation
    fig.add_trace(
        go.Scatter(
            x=df['indian_cpi'],
            y=df['overall_index'],
            mode='markers',
            name='Indian CPI',
            marker=dict(color='green', opacity=0.6)
        ),
        row=2, col=1
    )
    
    # Oil Prices vs Inflation
    fig.add_trace(
        go.Scatter(
            x=df['crude_oil_prices'],
            y=df['overall_index'],
            mode='markers',
            name='Oil Prices',
            marker=dict(color='purple', opacity=0.6)
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        title_text="Relationship between Inflation and Economic Drivers",
        template="plotly_white",
        showlegend=True
    )
    
    fig.show()

plot_economic_drivers()

In [8]:
# 5. Plot Volatility Analysis
def plot_volatility_analysis():
    """Plot volatility analysis using plotly"""
    # Calculate rolling standard deviation
    rolling_std = pd.DataFrame({
        'Overall': df['overall_index'].rolling(12).std(),
        'Food': df['food_and_beverage'].rolling(12).std(),
        'Non-food': df['non_food_and_services'].rolling(12).std()
    })
    
    fig = go.Figure()
    
    colors = ['blue', 'green', 'red']
    for col, color in zip(rolling_std.columns, colors):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=rolling_std[col],
                name=col,
                line=dict(color=color, width=2)
            )
        )
    
    fig.update_layout(
        title="Volatility in Inflation Components (12-month Rolling Std)",
        xaxis_title="Date",
        yaxis_title="Standard Deviation",
        template="plotly_white",
        height=500,
        showlegend=True
    )
    
    fig.show()

plot_volatility_analysis()

In [9]:
# 6. Plot Policy Impact
def plot_policy_impact():
    """Plot policy impact analysis using plotly"""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Policy Rate and Inflation', 'Money Supply and Inflation'],
        specs=[[{"secondary_y": True}], [{"secondary_y": True}]]
    )
    
    # Policy rate and inflation
    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['overall_index'],
            name='Overall Index',
            line=dict(color='blue')
        ),
        secondary_y=False,
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['policy_rate'],
            name='Policy Rate',
            line=dict(color='red')
        ),
        secondary_y=True,
        row=1, col=1
    )
    
    # Money supply and inflation
    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['overall_index'],
            name='Overall Index',
            line=dict(color='blue'),
            showlegend=False
        ),
        secondary_y=False,
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['broad_money_m2'],
            name='M2',
            line=dict(color='green')
        ),
        secondary_y=True,
        row=2, col=1
    )
    
    fig.update_layout(
        height=800,
        title_text="Monetary Policy Impact Analysis",
        template="plotly_white",
        showlegend=True
    )
    
    fig.show()

plot_policy_impact()

In [10]:
def plot_international_inflation_comparison():
    """Compare inflation trends across Nepal, India, China, and US"""
    # Create subplots
    fig = make_subplots(rows=2, cols=1,
                       subplot_titles=['Consumer Price Indices Comparison',
                                     'Month-over-Month Inflation Rates'],
                       vertical_spacing=0.15)
    
    # Colors for each country
    colors = {
        'Nepal': 'blue',
        'India': 'orange',
        'China': 'red',
        'US': 'green'
    }
    
    # Plot CPI levels
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['overall_index'],
            name='Nepal CPI',
            line=dict(color=colors['Nepal'], width=2)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['indian_cpi'],
            name='India CPI',
            line=dict(color=colors['India'], width=2)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['china_cpi'],
            name='China CPI',
            line=dict(color=colors['China'], width=2)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['usa_cpi'],
            name='US CPI',
            line=dict(color=colors['US'], width=2)
        ),
        row=1, col=1
    )
    
    # Plot MoM inflation rates
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['overall_index_mom'],
            name='Nepal MoM',
            line=dict(color=colors['Nepal'], width=2)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['indian_cpi_mom'],
            name='India MoM',
            line=dict(color=colors['India'], width=2)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['china_cpi_mom'],
            name='China MoM',
            line=dict(color=colors['China'], width=2)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['usa_cpi_mom'],
            name='US MoM',
            line=dict(color=colors['US'], width=2)
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text='International Inflation Comparison (2009-2024)',
        template='plotly_white',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Update axes
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Consumer Price Index", row=1, col=1)
    fig.update_yaxes(title_text="Month-over-Month Change (%)", row=2, col=1)
    
    # Add range slider
    fig.update_xaxes(rangeslider_visible=True, row=2, col=1)
    
    # Add grid
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128, 128, 128, 0.2)')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128, 128, 128, 0.2)')
    
    fig.show()


plot_international_inflation_comparison()

In [11]:
def plot_usd_exchange_rate():
    """Visualize the USD/NPR exchange rate over time."""
    
    fig = go.Figure()

    # Plot USD exchange rate
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['usd'],
            name='USD/NPR',
            line=dict(color='blue', width=2)
        )
    )

    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Update layout
    fig.update_layout(
        height=400,
        title_text='USD to NPR Exchange Rate (2009-2024)',
        template='plotly_white',
        showlegend=False,
        xaxis_title="Date",
        yaxis_title="USD/NPR",
        xaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    # Add annotation for the latest value and total change
    latest_date = df.index[-1]
    usd_current = df['usd'].iloc[-1]
    usd_change = ((df['usd'].iloc[-1] - df['usd'].iloc[0]) / df['usd'].iloc[0]) * 100
    
    fig.add_annotation(
        x=latest_date,
        y=usd_current,
        text=f"USD/NPR: {usd_current:.2f}<br>Total Change: {usd_change:.1f}%",
        showarrow=True,
        arrowhead=1
    )
    
    fig.show()


plot_usd_exchange_rate()


In [12]:
def plot_crude_oil_prices():
    """Visualize crude oil prices over time."""
    
    fig = go.Figure()

    # Plot Crude Oil Prices
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['crude_oil_prices'],
            name='Crude Oil (USD/barrel)',
            line=dict(color='red', width=2)
        )
    )

    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Update layout
    fig.update_layout(
        height=400,
        title_text='Crude Oil Prices (USD/barrel) (2009-2024)',
        template='plotly_white',
        showlegend=False,
        xaxis_title="Date",
        yaxis_title="USD/Barrel",
        xaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    # Add annotation for the latest value and total change
    latest_date = df.index[-1]
    oil_current = df['crude_oil_prices'].iloc[-1]
    oil_change = ((df['crude_oil_prices'].iloc[-1] - df['crude_oil_prices'].iloc[0]) / df['crude_oil_prices'].iloc[0]) * 100
    
    fig.add_annotation(
        x=latest_date,
        y=oil_current,
        text=f"Oil Price: ${oil_current:.2f}<br>Total Change: {oil_change:.1f}%",
        showarrow=True,
        arrowhead=1
    )
    
    fig.show()


plot_crude_oil_prices()


In [13]:
def plot_wages():
    """Visualize agricultural, industrial, and construction wages over time."""
    
    fig = go.Figure()

    # Plot Agricultural Wages
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['agricultural_wages'],
            name='Agricultural Wages',
            line=dict(color='seagreen', width=2)
        )
    )

    # Plot Industrial Wages
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['industrial_wages'],
            name='Industrial Wages',
            line=dict(color='violet', width=2)
        )
    )

    # Plot Construction Wages
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['construction_wages'],
            name='Construction Wages',
            line=dict(color='orange', width=2)
        )
    )

    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Update layout
    fig.update_layout(
        height=600,
        title_text='Wage Trends in Different Sectors (2009-2024)',
        template='plotly_white',
        xaxis_title="Date",
        yaxis_title="Wages (NPR)",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    # Add annotations for latest values and total changes
    latest_date = df.index[-1]

    for column, color, name in zip(
        ['agricultural_wages', 'industrial_wages', 'construction_wages'],
        ['seagreen', 'violet', 'orange'],
        ['Agricultural Wages', 'Industrial Wages', 'Construction Wages']
    ):
        latest_value = df[column].iloc[-1]
        change_percent = ((df[column].iloc[-1] - df[column].iloc[0]) / df[column].iloc[0]) * 100
        
        fig.add_annotation(
            x=latest_date,
            y=latest_value,
            text=f"{name}: {latest_value:.2f}<br>Total Change: {change_percent:.1f}%",
            showarrow=True,
            arrowhead=1,
            arrowcolor=color,
            font=dict(color=color)
        )
    
    fig.show()


plot_wages()


In [14]:
def plot_remittance_inflow():
    """Visualize remittance inflow over time."""
    
    fig = go.Figure()

    # Plot Remittance Inflow
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['remittance_inflow'],
            name='Remittance Inflow',
            line=dict(color='purple', width=2)
        )
    )

    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Update layout
    fig.update_layout(
        height=600,
        title_text='Remittance Inflow Over Time (2009-2024)',
        template='plotly_white',
        xaxis_title="Date",
        yaxis_title="Remittance Inflow (NPR)",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    # Add annotations for the latest value and total change
    latest_date = df.index[-1]
    latest_value = df['remittance_inflow'].iloc[-1]
    change_percent = ((df['remittance_inflow'].iloc[-1] - df['remittance_inflow'].iloc[0]) / df['remittance_inflow'].iloc[0]) * 100

    fig.add_annotation(
        x=latest_date,
        y=latest_value,
        text=f"Remittance: {latest_value:.2f}<br>Total Change: {change_percent:.1f}%",
        showarrow=True,
        arrowhead=1,
        arrowcolor="purple",
        font=dict(color="purple")
    )
    
    fig.show()


plot_remittance_inflow()


In [15]:
def plot_fiscal_indicators():
    """Visualize government expenditure, government revenue, and fiscal deficit."""
    
    fig = go.Figure()

    # Plot Government Expenditure
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['government_expenditure'],
            name='Government Expenditure',
            line=dict(color='blue', width=2)
        )
    )

    # Plot Government Revenue
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['government_revenue'],
            name='Government Revenue',
            line=dict(color='green', width=2)
        )
    )

    # Plot Fiscal Deficit
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['fiscal_deficit'],
            name='Fiscal Deficit',
            line=dict(color='red', width=2, dash='dash')  # Dashed line for deficit
        )
    )

    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Update layout
    fig.update_layout(
        height=600,
        title_text='Fiscal Indicators Over Time (2009-2024)',
        template='plotly_white',
        xaxis_title="Date",
        yaxis_title="Amount (NPR)",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    # Add annotations for the latest values
    latest_date = df.index[-1]
    indicators = {
        'Government Expenditure': ('government_expenditure', 'blue'),
        'Government Revenue': ('government_revenue', 'green'),
        'Fiscal Deficit': ('fiscal_deficit', 'red')
    }

    for name, (col, color) in indicators.items():
        latest_value = df[col].iloc[-1]
        fig.add_annotation(
            x=latest_date,
            y=latest_value,
            text=f"{name}: {latest_value:.2f}",
            showarrow=True,
            arrowhead=1,
            arrowcolor=color,
            font=dict(color=color)
        )
    
    fig.show()


plot_fiscal_indicators()


In [16]:
def plot_commercial_bank_lending_rates():
    """Visualize commercial bank lending rates over time."""
    
    fig = go.Figure()

    # Plot Commercial Bank Lending Rates
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['commercial_bank_lending_rates'],
            name='Commercial Bank Lending Rates',
            line=dict(color='purple', width=2)
        )
    )

    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Update layout
    fig.update_layout(
        height=500,
        title_text='Commercial Bank Lending Rates (2009-2024)',
        template='plotly_white',
        xaxis_title="Date",
        yaxis_title="Lending Rate (%)",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    # Add annotation for the latest value
    latest_date = df.index[-1]
    latest_rate = df['commercial_bank_lending_rates'].iloc[-1]
    
    fig.add_annotation(
        x=latest_date,
        y=latest_rate,
        text=f"Lending Rate: {latest_rate:.2f}%",
        showarrow=True,
        arrowhead=1,
        arrowcolor="purple",
        font=dict(color="purple")
    )
    
    fig.show()


plot_commercial_bank_lending_rates()


In [17]:
def plot_trade_data():
    """Visualize import values, export values, and trade deficit over time."""
    
    fig = go.Figure()

    # Plot Import Values
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['import_values'],
            name='Import Values',
            line=dict(color='red', width=2)
        )
    )

    # Plot Export Values
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['export_values'],
            name='Export Values',
            line=dict(color='green', width=2)
        )
    )

    # Plot Trade Deficit
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['trade_deficit'],
            name='Trade Deficit',
            line=dict(color='blue', width=2)
        )
    )

    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Update layout
    fig.update_layout(
        height=600,
        title_text='Trade Data: Imports, Exports, and Trade Deficit (2009-2024)',
        template='plotly_white',
        xaxis_title="Date",
        yaxis_title="Value (in million NPR)",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    # Add annotations for the latest values
    latest_date = df.index[-1]
    
    # Import Values
    latest_import = df['import_values'].iloc[-1]
    fig.add_annotation(
        x=latest_date,
        y=latest_import,
        text=f"Imports: {latest_import:.2f}M NPR",
        showarrow=True,
        arrowhead=1,
        arrowcolor="red",
        font=dict(color="red")
    )
    
    # Export Values
    latest_export = df['export_values'].iloc[-1]
    fig.add_annotation(
        x=latest_date,
        y=latest_export,
        text=f"Exports: {latest_export:.2f}M NPR",
        showarrow=True,
        arrowhead=1,
        arrowcolor="green",
        font=dict(color="green")
    )
    
    # Trade Deficit
    latest_deficit = df['trade_deficit'].iloc[-1]
    fig.add_annotation(
        x=latest_date,
        y=latest_deficit,
        text=f"Trade Deficit: {latest_deficit:.2f}M NPR",
        showarrow=True,
        arrowhead=1,
        arrowcolor="blue",
        font=dict(color="blue")
    )
    
    fig.show()


plot_trade_data()


In [18]:
def plot_food_price_indices():
    """Visualize Global Food Price Index and Food & Beverage Index."""
    
    fig = go.Figure()

    # Plot Global Food Price Index
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['global_food_price_index'],
            name='Global Food Price Index',
            line=dict(color='orange', width=2)
        )
    )

    # Plot Food and Beverage Index
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['food_and_beverage'],
            name='Food and Beverage',
            line=dict(color='purple', width=2)
        )
    )

    # Layout updates
    fig.update_layout(
        title="Global Food Price Index vs Food & Beverage Index (2009-2024)",
        xaxis_title="Date",
        yaxis_title="Index Value",
        template="plotly_white",
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis=dict(rangeslider_visible=True, showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    fig.show()


plot_food_price_indices()


In [19]:
def plot_production_indices():
    """Visualize Agricultural Production Index and Industrial Production Index."""
    
    fig = go.Figure()

    # Plot Agricultural Production Index
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['agricultural_production_index'],
            name='Agricultural Production Index',
            line=dict(color='green', width=2)
        )
    )

    # Plot Industrial Production Index
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['industrial_production_index'],
            name='Industrial Production Index',
            line=dict(color='orange', width=2)
        )
    )

    # Layout updates
    fig.update_layout(
        title="Agricultural vs Industrial Production Indices (2009-2024)",
        xaxis_title="Date",
        yaxis_title="Index Value",
        template="plotly_white",
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis=dict(rangeslider_visible=True, showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(128, 128, 128, 0.2)')
    )
    
    fig.show()
    
plot_production_indices()
